# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow==2.12.0 opencv-python mediapipe sklearn matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [2]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pandas as pd

# 2. Keypoints using MP Holistic

In [6]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [7]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [8]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [9]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [10]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

# 5. Collect Keypoint Values for Training and Testing

DataPerp_MP_ISL.ipynb

In [136]:



df_from_parquet = pd.read_parquet(r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Mp_Data\LandMarks")

display(df_from_parquet)

,Label,sample_type,Sample_Number,frame_shape,Frame_Number,Keypoints
0,are you free today,Video,0,"[1080, 1920, 3]",1.0,"[0.47676751017570496, 0.3017379641532898, -0.6..."
1,are you free today,Video,0,"[1080, 1920, 3]",2.0,"[0.47681325674057007, 0.30156728625297546, -0...."
2,are you free today,Video,0,"[1080, 1920, 3]",3.0,"[0.4772616922855377, 0.3015567362308502, -0.63..."
3,are you free today,Video,0,"[1080, 1920, 3]",4.0,"[0.477945476770401, 0.30148565769195557, -0.62..."
4,are you free today,Video,0,"[1080, 1920, 3]",5.0,"[0.47859013080596924, 0.30141815543174744, -0...."
...,...,...,...,...,...,...
51558,"you need a medicine, take this one",Video,4,"[1080, 1920, 3]",114.0,"[0.49276989698410034, 0.35694217681884766, -0...."
51559,"you need a medicine, take this one",Video,4,"[1080, 1920, 3]",115.0,"[0.4929289221763611, 0.3573005497455597, -0.69..."
51560,"you need a medicine, take this one",Video,4,"[1080, 1920, 3]",116.0,"[0.49306631088256836, 0.35789814591407776, -0...."
51561,"you need a medicine, take this one",Video,4,"[1080, 1920, 3]",117.0,"[0.49327489733695984, 0.3583262860774994, -0.6..."


In [143]:
grouped = df_from_parquet
#.groupby(['Label'])
#display(grouped)
Frame_count=grouped['Frame_Number'].max()
print(Frame_count)

192.0


In [159]:
df_from_parquet.Keypoints[100].shape

(258,)

In [89]:
keypoints_data=df_from_parquet.head(2)["Keypoints"]
keypoints_arrays = [np.array(keypoint) for keypoint in keypoints_data]

In [91]:
keypoints_arrays

[array([ 0.47676751,  0.30173796, -0.69633353,  0.99985111,  0.48993158,
         0.26554024, -0.6616351 ,  0.99970132,  0.49743682,  0.26715803,
        -0.66179639,  0.99966407,  0.50508267,  0.26875257, -0.661816  ,
         0.99963665,  0.46471313,  0.26568586, -0.66376728,  0.99977046,
         0.45617524,  0.26697224, -0.66398817,  0.99978107,  0.44817939,
         0.26809919, -0.66433334,  0.99982786,  0.51699769,  0.2926504 ,
        -0.42892987,  0.99959356,  0.4363426 ,  0.29160309, -0.43342102,
         0.99980718,  0.49241033,  0.34576923, -0.60737127,  0.99993289,
         0.46151373,  0.34452379, -0.60890973,  0.99996197,  0.59039629,
         0.54260284, -0.27551523,  0.99907529,  0.36381176,  0.52392256,
        -0.26729125,  0.99948043,  0.60979867,  0.82670069, -0.2548207 ,
         0.95624506,  0.34302178,  0.81141615, -0.21275458,  0.96439791,
         0.57155931,  0.99670261, -0.53785068,  0.69867325,  0.3932929 ,
         1.00709033, -0.47202536,  0.73338234,  0.5

In [ ]:
import pandas as pd

# Assuming df_from_parquet is your DataFrame and "Label" is a column in it
df_label = df_from_parquet["Label"].unique()
print(df_label)


# 6. Preprocess Data and Create Labels and Features

In [28]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [29]:
label_map = {label:num for num, label in enumerate(df_label)}

In [43]:
label_map

{'are you free today': 0,
 'bring water for me': 1,
 'can i help you': 2,
 'can you repeat that please': 3,
 'comb your hair': 4,
 'congratulations': 5,
 'could you please talk slower': 6,
 'do me a favour': 7,
 'do not abuse him': 8,
 'do not be stubborn': 9,
 'do not hurt me': 10,
 'do not make me angry': 11,
 'do not take it to the heart': 12,
 'do not worry': 13,
 'do you need something': 14,
 'go and sleep': 15,
 'had your food': 16,
 'he came by train': 17,
 'He is going into the room': 18,
 'he is on the way': 19,
 'he she is my friend': 20,
 'he would be coming today': 21,
 'help me': 22,
 'hi how are you': 23,
 'how are things': 24,
 'how can i help you': 25,
 'how can i trust you': 26,
 'how dare you': 27,
 'how old are you': 28,
 'i am (age)': 29,
 'i am afraid of that': 30,
 'i am crying': 31,
 'i am feeling bored': 32,
 'i am feeling cold': 33,
 'i am fine. thank you sir': 34,
 'i am hungry': 35,
 'i am in dilemma what to do': 36,
 'i am not really sure': 37,
 'i am really

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
y_test.shape

# 7. Build and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

In [ ]:
model.summary()

# 8. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

# 9. Save Weights

In [ ]:
model.save('action.h5')

In [ ]:
del model

In [ ]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# 11. Test in Real Time

In [ ]:
from scipy import stats

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()